This notebook shows how to compare 2D datasets from ROOT and McStas simulations 

- load McStas and ROOT data
- crop ROOT data 
- resample x-y grid to the smallest grid 
- normalize intensity
- choose centering points
- plot difference as 2D and 3D plots

**Note**:  The physical units were not considered when comparing the arrays.

In [ ]:
import os
import uproot
import scipp as sc
import plopp as pp
import mpltoolbox as tbx

import dataconfig  # to get paths to data
import numpy as np
import re

import scipy.interpolate
import ipywidgets
#import matplotlib
#matplotlib.rcParams['figure.max_open_warning'] = 0

%matplotlib widget

# Load McStas data

In [ ]:
# Output of McStas simulation
selected_filename = 'monitor_tx_DENEX.dat'

path_to_mcstas2D_file = os.path.join(dataconfig.data_mcstas, selected_filename)

assert os.path.isdir(dataconfig.data_mcstas), \
'The folder which should contain outputs of McStas simulation does not exist.'

assert os.path.isfile(path_to_mcstas2D_file), \
    'There is an issue with the chosen McStas 2D datafile'

In [ ]:
def extract_metadata_from_file(path_to_mcstas2D_file):
    dict_metadata = {}
    one_dimensional_data = False
    with open(path_to_mcstas2D_file, 'r') as file:
        for line in file:
            if 'xlabel' in line:
                dict_metadata['xlabel'] = ' '.join(line.rstrip().split()[-2:])
            elif 'ylabel' in line:
                dict_metadata['ylabel'] = ' '.join(line.rstrip().split()[-2:])
            elif 'position' in line:
                dict_metadata['position'] = line.rstrip().split()[-3:] 
            elif "array_1d" in line:
                one_dimensional_data = True
                type_array = line.rstrip()
                start = type_array.find('(') + 1
                end = type_array.find(')', start)
                dict_metadata['nx_value'], dict_metadata['ny_value'] = [int(item) for item in type_array[start:end].split(',')]
        
            elif "array_2d" in line:
                one_dimensional_data = False
                type_array = line.rstrip()
                start = type_array.find('(') + 1
                end = type_array.find(')', start)
                dict_metadata['nx_value'], dict_metadata['ny_value'] = [int(item) for item in type_array[start:end].split(',')]
            elif "limits" in line:
                if one_dimensional_data == False:
                    dict_metadata['x_min'], dict_metadata['x_max'], dict_metadata['y_min'], dict_metadata['y_max'] = [float(item) for item in re.findall('-?[\d]*[.][\d]+', line)]
                else:
                    dict_metadata['x_min'], dict_metadata['x_max']  = [float(item) for item in re.findall('[\d]*[.][\d]+', line)]
                    
    return dict_metadata

In [ ]:
metadata_denex = extract_metadata_from_file(path_to_mcstas2D_file)
metadata_denex

In [ ]:
data2d = np.genfromtxt(path_to_mcstas2D_file,
                       max_rows=int(metadata_denex['ny_value']))

#if position_x of component < 0, flip data along y axis 
# otherwise leave unchanged
if float(metadata_denex['position'][0]) < 0:
     data2d = np.flip(data2d, 0)

da_mcstas = sc.DataArray(data=sc.array(dims=['x', 'tof'], values=data2d),
                          coords={'tof': sc.linspace('tof', metadata_denex['x_min'], metadata_denex['x_max'], num=metadata_denex['nx_value'], unit='s'),
                                  'x': sc.linspace('x', metadata_denex['y_min'], metadata_denex['y_max'], num=metadata_denex['ny_value'], unit='m')
                                  })
da_mcstas

In [ ]:
pp.plot(da_mcstas, grid=True, title=selected_filename)

# Load ROOT data

In [ ]:
path_to_root_file = dataconfig.data_root  

assert os.path.isdir(path_to_root_file), \
    'The path to the folder which should contain ROOT files does not exist.'

In [ ]:
# open one ROOT file and extract only one 1D and one 2D dataset specified in dict_selected_dataset
# The 1D and 2D datasets are stored in a dictionary
# Note the vertical axis of 2D datasets is inverted

key_spectrum ='Spectrum03'
ROOTfile = 'Spectrum03_DENEX006_1_18-02-05_0000.root'
dir_with_data = 'Meas_3'
selected_dataset = 'H_TOF,X1-X2_User_2D2_dsp_after_run_3'

file_to_open = os.path.join(path_to_root_file, ROOTfile)

assert os.path.isfile(file_to_open), 'There is an issue with the chosen ROOT file'

# dictionary to store the x, y boundaries of selected 2D root data
root_x_y_boundaries = {}

with uproot.open(file_to_open)[dir_with_data] as myFile:
    for key in myFile.keys():

        # 2D contourplot
        if selected_dataset in str(key):
            key_name = myFile[key].name
            data2d_root =  np.flip(myFile[key].counts(False), 1).transpose()
            # create x- and y-axis
            xaxis = myFile[key].axis(axis=0).edges()[:-1]
            yaxis = myFile[key].axis(axis=1).edges()[:-1]
            
            # extract info about x, y axis (min, max and number of bins) 
            root_x_y_boundaries['x_min'] = min(xaxis)
            root_x_y_boundaries['x_max'] = max(xaxis)
            root_x_y_boundaries['y_min'] = min(yaxis)
            root_x_y_boundaries['y_max'] = max(yaxis) 
            
            # name_output_file = ROOTfile[:10] + "_" + key_name.replace(',','_') + "_inv_y"

## Crop ROOT array (top, bottom)
Remove top and bottom rows of the ROOT array.
The boundaries can be set by the user using the slider.

In [ ]:
da_root = sc.DataArray(data=sc.array(dims=['x', 'tof'], values=data2d_root.astype(float)),
                       coords={'tof': sc.array(dims=['tof'], values=xaxis, unit='s'),
                              'x': sc.array(dims=['x'], values=yaxis, unit='m')})
da_root

In [ ]:
ydim = 'x'

def add_mask(da, trunc_range):
    min_tr, max_tr = trunc_range  
    
    out = da.copy(deep=False)
    if ydim in out.masks:
        del out.masks[ydim]
    out.masks[ydim] = ((out.coords[ydim] > out.coords[ydim][max_tr]) | (out.coords[ydim] < out.coords[ydim][min_tr]))
    return out

in_node = pp.input_node(da_root)
in_node.name = 'Input node'

# slider
max_slider = da_root.sizes[ydim]
slider = ipywidgets.IntRangeSlider(value=[0, max_slider-1],
                                   min=0, 
                                   max=max_slider-1,
                                   description='x indices')

slider_node = pp.widget_node(slider)
slider_node.name = 'Slider node'

# add mask
add_mask_node = pp.node(add_mask)(in_node, trunc_range=slider_node)
add_mask_node.name = 'Show masks'

fig = pp.figure2d(add_mask_node)

ipywidgets.VBox([slider, fig])

In [ ]:
xselect_min, xselect_max = slider_node.request_data()
filtered_root = da_root['x', xselect_min:xselect_max]
filtered_root

In [ ]:
pp.plot(filtered_root, title=f"CROPPED {key_name}")

# Rebin to the smallest grid
The array with the finest grid is resampled to the coarser grid.

In [ ]:
print(f"Shape of ROOT data: {filtered_root.shape}")
print(f"Shape of McStas data: {metadata_denex['nx_value']}, {metadata_denex['ny_value']}")

In [ ]:
filtered_root

In [ ]:
# new_mat = rebin(cropped_array, data2d.shape, **root_x_y_boundaries)
new_mat_root = filtered_root.hist(tof=da_mcstas.shape[0]).hist(x=da_mcstas.shape[1]).transpose()
new_mat_root

In [ ]:
new_mat_mcstas = da_mcstas.hist(tof=da_mcstas.shape[0]).hist(x=da_mcstas.shape[1]).transpose()

In [ ]:
pp.plot(new_mat_root, title='Resampled ROOT 2D array to different grid')

In [ ]:
print(f"Shape of reshaped ROOT data: {new_mat_root.shape}, and binned McStas data: {new_mat_mcstas.shape}")

# Rescale matrices' intensities
In order to compare the 2 matrices, the intensity has to be rescaled to the same values.

In [ ]:
rescaled_root = new_mat_root.copy()
rescaled_root *= 256/(new_mat_root.max() - new_mat_root.min())

rescaled_mcstas = new_mat_mcstas.copy()
rescaled_mcstas *= 256/(new_mat_mcstas.max() - new_mat_mcstas.min())

In [ ]:
print(f"""Shape of modified ROOT matrix: {rescaled_root.shape}
Shape of modified McStas matrix: {rescaled_mcstas.shape}
Max intensity of modified ROOT matrix: {sc.max(rescaled_mcstas).value}
Max intensity of modified ROOT matrix: {sc.max(rescaled_root).value}""")

# Choice of point to align arrays

TO DO: limit to one point per data array

In [ ]:
rescaled_mcstas.coords['tof'].min()

In [ ]:
pm = pp.plot(rescaled_mcstas, title='McStas')
pr = pp.plot(rescaled_root, title='ROOT')

points_root = tbx.Points(ax=pr.canvas.ax, ms=10, marker='+', mec='r')
points_mcstas = tbx.Points(ax=pm.canvas.ax, ms=10, marker='+', mec='r')

def change_point_shape(point_selected):
    point_selected.marker = 'o'
    
def reset_point_shape(point_released):
    point_released.marker = '+' 
    
def only_one_point_root(point):
    # pr.ax.set_title(str(len(points_root.children)))
    # pr.ax.set_title(str(len(point.associated.children)))
    if len(points_root.children) > 1:
        point.remove()
        # for i in range(1, len(points_root.children)):
        #     points_root.children[i].remove()
    
    # points_root.children[0].xy   
    # print(dir(change['artist'].associated))
    # change['artist'].associated = l

def only_one_point_mcstas(point):
    if len(points_mcstas.children) > 1:
        point.remove()
        
points_root.on_vertex_press(change_point_shape)
points_root.on_vertex_release(reset_point_shape)
points_root.on_create(only_one_point_root)

points_mcstas.on_vertex_press(change_point_shape)
points_mcstas.on_vertex_release(reset_point_shape)
points_mcstas.on_create(only_one_point_mcstas)

pp.widgets.Box([[pr, pm]])

# Calculate difference of arrays

In [ ]:
# Define difference array
x0_root = points_root.children[0].x
y0_root = points_root.children[0].y

x0_mcstas = points_mcstas.children[0].x
y0_mcstas = points_mcstas.children[0].y

offset_x = x0_root - x0_mcstas
offset_y = y0_root - y0_mcstas

print(x0_root, y0_root, x0_mcstas, y0_mcstas)

assert rescaled_root.coords['tof'].unit == rescaled_mcstas.coords['tof'].unit, "ROOT and McStas matrices have different tof units"
assert rescaled_root.coords['x'].unit == rescaled_mcstas.coords['x'].unit, "ROOT and McStas matrices have different x units"

unit_tof = rescaled_root.coords['tof'].unit
unit_x = rescaled_root.coords['x'].unit

rescaled_rootc = rescaled_root.copy()
rescaled_mcstasc = rescaled_mcstas.copy()

rescaled_rootc.coords['tof'] += max(0, offset_x) * unit_tof
rescaled_rootc.coords['x'] += max(0, offset_y) * unit_x

rescaled_mcstasc.coords['tof'] += min(0, offset_x) * unit_tof
rescaled_mcstasc.coords['x'] += min(0, offset_y) * unit_x

# we discard the coordinates to only keep the 2D "intensity" values
diff_arrays = rescaled_rootc.data.values - rescaled_mcstasc.data.values
diff_arrays

In [ ]:
# info about difference array: size, min and max intensities
print(f"""Shape of difference array: {diff_arrays.shape}
Max and min intensities of difference array: {np.max(diff_arrays)}, {np.min(diff_arrays)}""")

In [ ]:
da_diff  = sc.DataArray(data=sc.array(dims=['x', 'tof'], values=diff_arrays),
                          coords={'tof': sc.linspace('tof', 0, diff_arrays.shape[0]-1, num=diff_arrays.shape[0], unit='dimensionless'),
                                  'x': sc.linspace('x', 0, diff_arrays.shape[1]-1, num=diff_arrays.shape[1], unit='dimensionless')
                                  })
da_diff

In [ ]:
p_diff_2d = pp.plot(da_diff)

In [ ]:
# da_diff_3d = da_diff.copy()
# da_diff_3d.coords['intensity'] = da_diff.data# .flatten(to='tofx')
# da_diff_3d

nb_rows = len(da_diff.coords['tof']) * len(da_diff.coords['x'])
tof_3d = sc.zeros(dims=['row'], shape=[nb_rows])
x_3d = sc.zeros(dims=['row'], shape=[nb_rows])
indx = 0

for i in range(len(da_diff.coords['tof'])):
    for j in range(len(da_diff.coords['x'])):
        tof_3d['row', indx] = da_diff.coords['tof'][i].value
        x_3d['row', indx] = da_diff.coords['x'][j].value
        
        indx +=1

In [ ]:
da_diff_3d = sc.DataArray(data=sc.array(dims=['row'], values=diff_arrays.flatten()),
                          coords={'tof': tof_3d,
                                   'x': x_3d,
                                   'intensity': sc.array(dims=['row'], values=da_diff.data.values.flatten())
                                  })
da_diff_3d

In [ ]:
p_diff_3d = pp.scatter3d(da_diff_3d, x='tof', y='x', z='intensity', figsize=(450,450))
p_diff_3d.children[0].canvas.move_camera((280, -280, 400))
p_diff_3d

In [ ]:
pp.widgets.Box([[pr, pm], [p_diff_3d, p_diff_2d]])